<a href="https://colab.research.google.com/github/girish342/flaskSaaS/blob/master/making_environment_for_dqn_2_nd_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np  # for array stuff and random
from PIL import Image  # for creating visual of our env
import cv2  # for showing our visual live
import matplotlib.pyplot as plt  # for graphing our mean rewards over time
import pickle  # to save/load Q-Tables
from matplotlib import style  # to make pretty charts because it matters.
import time  # using this to keep track of our saved Q-Tables.

style.use("ggplot")  # setting our style!

In [0]:
# we r gonna create our own environment where square grid will be 10 X 10 size because it will take only 15 mb of meomory if we use 20 X 20 then it wil take 195 mb
# we r gonna define hyperparameters here befor defining our environment

SIZE = 10

HM_EPISODES = 25000
MOVE_PENALTY = 1  # feel free to tinker with these,these r random values by sentdex
ENEMY_PENALTY = 300  # feel free to tinker with these!
FOOD_REWARD = 25  # feel free to tinker with these!
epsilon = 0.5  # randomness
EPS_DECAY = 0.9999  # Every episode will be epsilon*EPS_DECAY,it converts learnig into earning 
SHOW_EVERY = 1000  # how often to play through env visually, weather we r going in right direction or not 


start_q_table = None # weather we have q table or not, there is no need to have q table if we dont have it

LEARNING_RATE = 0.1 # what step to be taken to achieve GRADIENT DESCENT
DISCOUNT = 0.95   # its tells what is future vs current reward


# we r giving numbers to the players
PLAYER_N = 1  # player key in dict
FOOD_N = 2  # food key in dict
ENEMY_N = 3  # enemy key in dict

# we r using dictinory for colors why we using dictionary(because it gives colors to respected player,food,enenmy) ,it contains bgr = blue,green,red.and giving numbers to colors 
d = {1: (255, 175, 0),  # blueish color(1 = blue,2 = green)
     2: (0, 255, 0),  # green
     3: (0, 0, 255)}  # red





In [9]:
# all hyperparmaters, numbers, colors,dependencies are defined
# we will now create environment by specifying class which is oop 
#class blob, creating init method so that we can interact with objects with class.
# i have to get intuition which class to use , how to use, when to use and happy learning and changing world for bertterment of humanity.
# string class to be used in oop to debug the process which i dont know which return the function along with x and y.
# __init__ constructs class, it is gateway for class to interact with its own attributes like property, tax/gov, own money




class Blob:
  def __init__(self):
    self.x = np.random.randint(0,SIZE)  # blob is square,when we start the environment it will start randomly as random.rand.
    self.y = np.random.randint(0,SIZE)  # x = input, y = output ;input dim = o to size which is 10 x 10
    
  def __string__(self):                 # we are using string because to debug according to sentdex, which i dont get it.
    return f"{self.x}, {self.y}"        # what to debug and how to debug i dont know just according to sentdex i have to research it
  
  
  def __sub__(self,other):            # we r crating sub(vajabaki) method so that we can substract from one blob to another blob        
    return(self.x-other.x, self.y-other.y) # first sub = player - food;2nd sub = player - enemy; it is called as opertor overloading which i dont know; 

  
# we r crating random action where the agent will move diagonally,up,low. we can add the actions if we want to add it 
# but magical thing about q network is that WE R NOT SPECIFYING THE ACTIONS IT WILL FIGURE OUT ON ITS OWN THATS POWER OF Q LEARINGING
# this is your typical function no need to use __init__ method
# without permision of self, class cannot interact with its own attributes. for eg courts give permission to father to met his kids.
# these r 4 actions (up,down,diagonal right,left)

  def action(self,choice):  
    
    if choice == 0:
      self.move(x=1,y=1)
      
    if choice == 1:
      self.move(x=-1,y=-1)
      
      
    if choice == 2:
      self.move(x=-1,y=1)
      
    if choice == 3:
      self.move(x=1,y=-1)
      
      
# self is hidden parameter in other language but in python we have to give respect to self in special way to felicitate it specially
# self rationlize parameters which we r passing to the class otherwisse class should have raised an error if we directly pass to it
# in move method we r moving x and y randomly by putting them false in move def intially so that we can explore the environment.
# python its boolian (if not x means  random ) its x or random 
# we r moving randomly, if we r not getting values for x and y. these are possibilites that s why we have not used __init__which is fix method.
# in __init__ method everthing is fixed.it canot be used in random method

def move(self, x=False, y=False):  # we have to move x and y randomly so that we r making them false we can do any thing here so that  its called as oop
    if not x:
      self.x +=  np.random.randint(-1, 2)  # why it is (-1, 2) i didnot get it.
      
    else:
      self.x += x  # if it doesnot move randomly move in desired direction.
      
# its python bullian y if not y = random.

    if not y:
      self.y +=  np.random.randint(-1, 2)
      
    else:
      self.y += y
      
# x cannot less than 0 if it is less than 0 then make it 0(first 2 lines).its oop we can do anything here
# x should not be more than size if it so then make it of size,size-1= 0 to 9 we r giving bounries to x.
    if self.x < 0:
      self.x = 0
    if self.x > SIZE-1:
      self.x = SIZE-1 
      
# same about y it shouldnot go oversize and undersize(it shouldnot cross boundries)    
    if self.y < 0:
      self.y= 0
    if self.y > SIZE-1:
      self.y = SIZE-1 
      
      
# we r checking all parameters here our class object construction is done 
# write now we will make q table
# subject understanding is 80%                                           26 june 2019

player = Blob()
food = Blob()
enemy = Blob()


print(player)
print(food)
print(player-food)
player.move()
print(player-food)
player.action(2)
print(player-food)     


# upto now we have created dependancies,defined hyperparameters,createdd environment(action,movement,limitations,defining player.food.enemy.


if start_q_table is None:                 # if we dont have the q table we have to create one
  q_table = {}                            # it is dictionary because it provides relation bet state and action thats why it is dictionary
  for i in range(-SIZE+1, SIZE):           # we are creating q table with range of sizes,from negative to positive max upto 9
    for ii in range(-SIZE+1, SIZE):        # first i = player,2nd i = food, 3rd i = enenmy, 4th i = food-player
      for iii in range(-SIZE+1, SIZE):
        for iiii in range(-SIZE+1, SIZE): # adding all above values to q table which is list of tuple of tuples
          q_table[((i,ii),(iii,iiii))] = [np.random.uniform(-5,0) for i in range (4)] # we have 4 actions so observation space should 4 so its includes bet 0 to -5, but why these values i dont know
          
else:
  with open(start_q_table,"rb")as f: # if we dont have q table then we save it with pickle and load it q_tablel.
    q_table = pickle.load(f)
    
    
# upto now we have crated environment,q table and aslo loaded the q table now we start iteration through episodes.

episode_rewards = []

for episode in range(HM_EPISODES):  # we r checking episode reward by reintializing player,food,enemy
  player = Blob()
  food = Blob()
  enemy = Blob()
  
  
  if episode % SHOW_EVERY == 0:
    print(f"on #{episode}, epsilon is {epsilon}")  # printing every episode and epsilon to see where we r
    print(f"{SHOW_EVERY} ep mean: {np.mean(episode_rewards[-SHOW_EVERY:])}") # we r gonna display min episode reward
    
    show = True
    
  else:
    show = False
    
    
    episode_reward = 0  # from starting of state episode reward will be 0.
    
    for i in range(200):  # we r taking 200 steps in one episode thats we wrote episode_reward not episode_rewards.
      obs = (player-food,player-enemy) # we take this observation from sub method which is operator overloding which is i dont know
      #rint(obs)
      
      if np.random.random() > epsilon:  # if exploitation is > exploration
        action = np.argmax(q_table[obs]) # get max action 
        
      else:
        action = np.random.randint(0,4)  # either we get max action or predefined 4 actions
        
      player.action(action)
      
      #enemy.move  # should not move intially because it complecates the thing acc to sentdex
      #food.move
      
# after taking actions we will decide what is reward whether its penulty or reward defing parameters for penulty.      
      if player.x == enemy.x and player.y == enemy.y:
        reward = -ENEMY_PENULTY   # its negative otherwise player will love the enemy if it is not punished,we have to avoid enemy
      
        
      elif player.x == food.x and player.y == food.y:
        reward = FOOD_REWARD
        
      else:
        reward = -MOVE_PENULTY  # every failure is punished thats life we have to be succeessful otherwise pay the penulty.
        
        
      new_obs = (player-food, player-enemy)
      
      max_future_q = np.max(q_table(new_obs))
      current_q = q_table[obs][action]
      
      if reward == FOOD_REWARD:
        new_q = FOOD_REWARD
        
        
# max future_q is calculated on current_q value because we want future_q value max(it is bet 0 and 1).
# discount = how much we care about future rewards rather than immeditate rewards we need both rewards because immediate reward = sustain, future_reward = to grow
# new_q = (1-alpha) * current_q + alpha * learned_value(reward+ discount*max_future_q)
# we use learining rate so much because its decides gradient descent which backpropagetes and modify weights so that we have minimum error and maximum result.
# this equation applies to life because how we play with our life how we balance it its completely here it decides our present condition,next line
# this equation describes current situation(penulties,rewards) and learning future rewards future its brutal work hard my boy.

      else:
    
        new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q) # its optimala q value for optimal state and optimal action, it is based on learned value on preivious episodes current q value..
        
      new_q = q_table[obs][action]
 
# right now q learing is completed we have to see what we have done and graph some matrix

      if show:
        env = np.zeros((SIZE, SIZE, 3), dtype=np.uint8)  # i m wrapping img with int 8,converting from bgr to rgb.
        env[food.x][food.y] = d[FOOD_N]
        env[player.x][player.y] = d[PLAYER_N]
        env[enemy.x][enemy.y] = d[ENEMY_N]
        img = Image.fromarray[env,'RGB']   # image in bgr shape we r converting into rgb
        img = img.resize((300,300))  # size of imge is 10 x 10 converting size into 300 because we cant see properly
        cv2.imshow = (("image",np.array(img)))
        if reward == FOOD_REWARD or reward == -ENEMY_PENULTY:  # if player got food or hit enemy we will pause
          if cv2.waitKey(500) & 0xFF == ord('q'):             # 500 miliseconds pause and break with q key
            break
        
        else:
          if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
      episode_reward += reward    # if this happens we end episode and start new episode
      if reward == FOOD_REWARD or reward == -ENEMY_PENALTY:
        break
        
        
    episode_rewards.append(episode_reward)
    epsilon *= EPS_DECAY
      

moving_avg = np.convolve(episode_rewards,np.ones((SHOW_EVERY,)) / SHOW_EVERY, mode='valid')
plt.plot([i for i in range(len(moving_avg))],moving_avg)
plt.ylabel(f"Reward{SHOW_EVERY}ma")
plt.xlabel("episode #")
plt.plot()


with open(f"q_table-{int(time.time())}.pickle","wb") as f:
  pickle.dump(q_table,f)
            
          
        
        
        
        
      
      
      
      
      
        
        
      
        
        
      
      
    
    

  
  
  
 
    
   
  







  
      




    
    
      
  
  
  
  
  
  
  
  
  
  
  
  
     
    
    
    
    
    
 
  
    
 
 


(-4, -2)


AttributeError: ignored